In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20201108.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,11/4/2020 7:19:45,Zardoz (1974),Solaris (1972),Face/Off (1997),Phase IV (1974),Alien (1979),OOPS! ALL NICOLAS CAGE...AGAIN,DISGUISES,MID-LIFE CRISIS,"WHERE ACTORS, PERHAPS NICHOLAS CAGE AND JOHN T...",OOPS! ALL RICK MORANIS,No Theme/Free-for-all
1,11/4/2020 7:20:18,Solaris (1972),Phase IV (1974),Zardoz (1974),NaN,NaN,OOPS! ALL NICOLAS CAGE...AGAIN,MID-LIFE CRISIS,DISGUISES,"WHERE ACTORS, PERHAPS NICHOLAS CAGE AND JOHN T...",OOPS! ALL RICK MORANIS,No Theme/Free-for-all
2,11/4/2020 7:23:36,Face/Off (1997),Solaris (1972),Phase IV (1974),Alien (1979),Zardoz (1974),"WHERE ACTORS, PERHAPS NICHOLAS CAGE AND JOHN T...",OOPS! ALL NICOLAS CAGE...AGAIN,DISGUISES,MID-LIFE CRISIS,OOPS! ALL RICK MORANIS,No Theme/Free-for-all
3,11/4/2020 8:34:55,Alien (1979),Zardoz (1974),Face/Off (1997),Phase IV (1974),NaN,OOPS! ALL RICK MORANIS,DISGUISES,OOPS! ALL NICOLAS CAGE...AGAIN,NaN,NaN,NaN
4,11/4/2020 15:34:10,Alien (1979),Zardoz (1974),Face/Off (1997),Phase IV (1974),Solaris (1972),OOPS! ALL RICK MORANIS,OOPS! ALL NICOLAS CAGE...AGAIN,"WHERE ACTORS, PERHAPS NICHOLAS CAGE AND JOHN T...",DISGUISES,MID-LIFE CRISIS,No Theme/Free-for-all
5,11/4/2020 17:11:20,Zardoz (1974),Face/Off (1997),NaN,NaN,NaN,OOPS! ALL RICK MORANIS,NaN,NaN,NaN,NaN,NaN
6,11/6/2020 8:19:18,Face/Off (1997),Zardoz (1974),Phase IV (1974),Solaris (1972),Alien (1979),OOPS! ALL NICOLAS CAGE...AGAIN,MID-LIFE CRISIS,DISGUISES,OOPS! ALL RICK MORANIS,"WHERE ACTORS, PERHAPS NICHOLAS CAGE AND JOHN T...",No Theme/Free-for-all
7,11/6/2020 10:04:41,Zardoz (1974),Phase IV (1974),Face/Off (1997),Solaris (1972),Alien (1979),OOPS! ALL NICOLAS CAGE...AGAIN,OOPS! ALL RICK MORANIS,MID-LIFE CRISIS,"WHERE ACTORS, PERHAPS NICHOLAS CAGE AND JOHN T...",No Theme/Free-for-all,DISGUISES


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Zardoz (1974)', 'Solaris (1972)', 'Face/Off (1997)',
       'Phase IV (1974)', 'Alien (1979)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Zardoz (1974)': <Candidate('Zardoz (1974)')>,
 'Solaris (1972)': <Candidate('Solaris (1972)')>,
 'Face/Off (1997)': <Candidate('Face/Off (1997)')>,
 'Phase IV (1974)': <Candidate('Phase IV (1974)')>,
 'Alien (1979)': <Candidate('Alien (1979)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Zardoz (1974), Solaris (1972), Face/Off (1997), Phase IV (1974), Alien (1979))>,
 <Ballot(Solaris (1972), Phase IV (1974), Zardoz (1974))>,
 <Ballot(Face/Off (1997), Solaris (1972), Phase IV (1974), Alien (1979), Zardoz (1974))>,
 <Ballot(Alien (1979), Zardoz (1974), Face/Off (1997), Phase IV (1974))>,
 <Ballot(Alien (1979), Zardoz (1974), Face/Off (1997), Phase IV (1974), Solaris (1972))>,
 <Ballot(Zardoz (1974), Face/Off (1997))>,
 <Ballot(Face/Off (1997), Zardoz (1974), Phase IV (1974), Solaris (1972), Alien (1979))>,
 <Ballot(Zardoz (1974), Phase IV (1974), Face/Off (1997), Solaris (1972), Alien (1979))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate          Votes  Status
---------------  -------  --------
Zardoz (1974)          3  Hopeful
Face/Off (1997)        2  Hopeful
Alien (1979)           2  Hopeful
Solaris (1972)         1  Rejected
Phase IV (1974)        0  Rejected

FINAL RESULT
Candidate          Votes  Status
---------------  -------  --------
Zardoz (1974)          4  Elected
Face/Off (1997)        2  Rejected
Alien (1979)           2  Rejected
Solaris (1972)         0  Rejected
Phase IV (1974)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                                                              Votes  Status
-------------------------------------------------------------------  -------  --------
OOPS! ALL NICOLAS CAGE...AGAIN                                             4  Elected
OOPS! ALL RICK MORANIS                                                     3  Rejected
WHERE ACTORS, PERHAPS NICHOLAS CAGE AND JOHN TRAVOLTA, TRADE FACES.        1  Rejected
DISGUISES                                                                  0  Rejected
MID-LIFE CRISIS                                                            0  Rejected
No Theme/Free-for-all                                                      0  Rejected

